In [137]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_csv('/users/youcefdjeddar/downloads/meteorite-landings.csv')
top_10 = data['recclass'].value_counts().index.tolist()[:10]
df = data[data['recclass'].isin(top_10)]
df.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"
7,Agen,392,Valid,H5,30000.0,Fell,1814.0,44.21667,0.61667,"(44.216670, 0.616670)"
8,Aguada,398,Valid,L6,1620.0,Fell,1930.0,-31.60000,-65.23333,"(-31.600000, -65.233330)"


In [38]:
df_with_dummies = pd.get_dummies(df, columns = ['recclass', 'fall'])

In [65]:
with pd.option_context('mode.use_inf_as_null', True):
   df_with_dummies = df_with_dummies.dropna()
df_with_dummies.columns

Index(['name', 'id', 'nametype', 'mass', 'year', 'reclat', 'reclong',
       'GeoLocation', 'recclass_CM2', 'recclass_H4', 'recclass_H4/5',
       'recclass_H5', 'recclass_H6', 'recclass_L4', 'recclass_L5',
       'recclass_L6', 'recclass_LL5', 'recclass_LL6', 'fall_Fell',
       'fall_Found'],
      dtype='object')

In [104]:
X = df_with_dummies[['mass', 'year', 'fall_Fell', 'fall_Found', 'reclat', 'reclong']]
y = df_with_dummies[['recclass_CM2', 'recclass_H4', 'recclass_H4/5',
       'recclass_H5', 'recclass_H6', 'recclass_L4', 'recclass_L5',
       'recclass_L6', 'recclass_LL5', 'recclass_LL6']]

In [105]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [144]:
knn = KNeighborsClassifier(n_neighbors = 10000)

In [145]:
knn.fit(X_train_scaled, y_train)

/Users/youcefdjeddar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10000, p=2,
           weights='uniform')

In [146]:
print("Training set score: {:.2f}".format(knn.score(X_train_scaled, y_train)))
print("Test set score: {:.2f}".format(knn.score(X_test_scaled, y_test)))

Training set score: 0.12
Test set score: 0.12


In [134]:
lsv = LinearSVC(C = 1, random_state = 0).fit(X_train_scaled, y_train)
print("Training set score: {:.2f}".format(lsv.score(X_train_scaled, y_train)))
print("Test set score: {:.2f}".format(lsv.score(X_test_scaled, y_test)))

/Users/youcefdjeddar/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training set score: 0.25
Test set score: 0.09


In [147]:
X1 = df_with_dummies[['recclass_CM2', 'recclass_H4', 'recclass_H4/5',
       'recclass_H5', 'recclass_H6', 'recclass_L4', 'recclass_L5',
       'recclass_L6', 'recclass_LL5', 'recclass_LL6']]
y1 = df_with_dummies[['mass']]

In [148]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1 ,random_state = 0)
linreg = LinearRegression().fit(X1_train, y1_train)

In [149]:
linreg = LinearRegression().fit(X1_train, y1_train)

In [150]:
linreg.score(X1_test, y1_test)

-0.004235899225622397

In [151]:
linreg.score(X1_train, y1_train)

0.0006353418622160323

In [152]:
poly = PolynomialFeatures(degree=2)
X_F1_poly = poly.fit_transform(X1)
X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y1, random_state = 0)
linreg = LinearRegression().fit(X1_train, y1_train)

In [153]:
linreg.score(X1_test, y1_test)

-0.004235899225622397

In [154]:
linreg.score(X1_train, y1_train)

0.0006353418622160323

In [155]:
ridge = Ridge(alpha = 200, random_state = 0).fit(X1_train, y1_train)
print("Training set score: {:.2f}".format(ridge.score(X1_train, y1_train)))
print("Test set score: {:.2f}".format(ridge.score(X1_test, y1_test)))

Training set score: 0.00
Test set score: -0.00


In [156]:
scaler = MinMaxScaler()
X1_train_scaled = scaler.fit_transform(X1_train)
X1_test_scaled = scaler.fit_transform(X1_test)
lasso = Lasso(alpha = 200, random_state = 0).fit(X1_train_scaled, y1_train)
print("Training set score: {:.2f}".format(lasso.score(X1_train_scaled, y1_train)))
print("Test set score: {:.2f}".format(lasso.score(X1_test_scaled, y1_test)))

Training set score: 0.00
Test set score: -0.00
